In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import plotly.express as px

warnings.filterwarnings('ignore')

Lecture des données

In [2]:
path_to_data = r"C:\Users\meghn\Documents\Dauphine\Master\M2 Finance Quantitative\Semestre 2\Machine Learning\data\\"
train_home_team_statistics_df = pd.read_csv(path_to_data+'Train_Data/train_home_team_statistics_df.csv', index_col=0)
train_away_team_statistics_df = pd.read_csv(path_to_data+'Train_Data/train_home_team_statistics_df.csv', index_col=0)
train_scores = pd.read_csv(path_to_data+'Y_train_1rknArQ.csv', index_col=0)

train_home = train_home_team_statistics_df.iloc[:,2:]
train_away = train_away_team_statistics_df.iloc[:,2:]

train_home.columns = 'HOME_' + train_home.columns
train_away.columns = 'AWAY_' + train_away.columns

train_data =  pd.concat([train_home,train_away],join='inner',axis=1)
train_scores = train_scores.loc[train_data.index]

train_data = train_data.replace({np.inf:np.nan,-np.inf:np.nan})


On essaie de sélectionner les features qui nous sont pertinentes.

In [3]:
test1 = train_home_team_statistics_df.reset_index().merge(train_scores.reset_index(), on="ID").set_index('ID')

In [4]:
corr_home = test1.iloc[:,2:].corr(numeric_only=False)
cols_to_keep = train_scores.columns.to_list()

In [7]:
# sorted_mat = corr_home.unstack().sort_values() 
upper_corr_mat = corr_home.where( 
    np.triu(np.ones(corr_home.shape), k=1).astype(bool)) 
unique_corr_pairs = upper_corr_mat.unstack().dropna() 
sorted_mat = unique_corr_pairs.sort_values() 

In [11]:
sorted_mat.to_excel("matrice_corr_home.xlsx")

In [12]:
shots = [col for col in test1 if 'SHOTS' in col]

In [23]:
fig = px.imshow(corr_home[shots + cols_to_keep].loc[shots + cols_to_keep], contrast_rescaling='infer')
fig.show()

on va essayer de trouver quels champs sont +/- facteurs de succès à domicile

In [4]:
train_away_team_statistics_df

,LEAGUE,TEAM_NAME,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,...,TEAM_YELLOWCARDS_5_last_match_std,TEAM_REDCARDS_5_last_match_std,TEAM_OFFSIDES_5_last_match_std,TEAM_ATTACKS_5_last_match_std,TEAM_PENALTIES_5_last_match_std,TEAM_SUBSTITUTIONS_5_last_match_std,TEAM_BALL_SAFE_5_last_match_std,TEAM_DANGEROUS_ATTACKS_5_last_match_std,TEAM_INJURIES_5_last_match_std,TEAM_GOALS_5_last_match_std
ID,,,,,,,,,,,,,,,,,,,,,
0,Ligue 1,Toulouse,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,...,3.0,0.0,6.0,0.0,10.0,8.0,7.0,2.0,4.0,3.0
1,Ligue 2,Brest,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,...,4.0,0.0,4.0,3.0,10.0,0.0,1.0,2.0,8.0,4.0
2,Serie A,Sampdoria,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,...,4.0,5.0,6.0,3.0,6.0,7.0,2.0,3.0,2.0,4.0
3,League One,Coventry City,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,...,4.0,0.0,1.0,8.0,8.0,5.0,5.0,5.0,NaN,6.0
4,Premier League,Wolverhampton Wanderers,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,...,1.0,0.0,2.0,5.0,8.0,7.0,2.0,6.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12298,League One,Southend United,4.0,2.0,3.0,4.0,7.0,4.0,4.0,3.0,...,4.0,0.0,2.0,5.0,0.0,5.0,0.0,0.0,NaN,4.0
12299,Liga Portugal,Boavista,4.0,2.0,3.0,1.0,5.0,1.0,1.0,9.0,...,6.0,10.0,7.0,2.0,0.0,0.0,3.0,10.0,6.0,1.0
12300,Bundesliga,Schalke 04,4.0,3.0,5.0,3.0,5.0,1.0,1.0,6.0,...,7.0,8.0,0.0,7.0,0.0,0.0,3.0,1.0,3.0,2.0


In [6]:
train_scores

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
0,0,0,1
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0
...,...,...,...
12298,0,0,1
12299,0,0,1
12300,0,0,1
